In [1]:
struct F{p, T<:Integer} <: Integer
    a::T
    F{p, T}(a::Integer) where {p, T<:Integer} = new{p, T}(mod(T(a), p))
end
F{p}(a::Integer) where p = F{p, typeof(a)}(mod(a, p))
F{p, S}(x::F{p, T}) where {S<:Integer, p, T<:Integer} = F{p, S}(x.a)

Base.promote_rule(::Type{F{p, T}}, ::Type{S}) where {p, T<:Integer, S<:Integer} =
    F{p, promote_type(T, S)}

Base.zero(::Type{F{p, T}}) where {p, T<:Integer} = F{p}(mod(zero(T), p))
Base.one(::Type{F{p, T}}) where {p, T<:Integer} = F{p}(mod(one(T), p))
for op in (:-, :+)
    @eval Base.$op(x::F{p}) where p = F{p}(mod($op(x.a), p))
end
for op in (:-, :+, :*)
    @eval Base.$op(x::F{p}, y::F{p}) where p = F{p}(mod($op(x.a, y.a), p))
end
Base.inv(x::F{p}) where p = F{p}(invmod(x.a, p))
Base.:/(x::F{p}, y::F{p}) where p = x * inv(y)
Base.:\(x::F{p}, y::F{p}) where p = inv(x) * y
Base.:(==)(x::F{p}, y::F{p}) where p = x.a == y.a
Base.:<(x::F{p}, y::F{p}) where p = x.a < y.a

Base.show(io::IO, x::F{p}) where p = print(io, "F", p, '(', x.a, ')')

F7 = F{7, BigInt}
x, y = F7(10), F7(-2)
@show(x, y, zero(x), one(x), +x, -x, x + y, x - y, x * y, x / y, x \ y, x^3, x^-5, x == F7(3), x == 3)
println()
A = F7[1 2; 3 4]
@show(A, 4A, A/4)
println()
using LinearAlgebra
L, U = lu(A, NoPivot())
@show(L, U, L * U, det(A), inv(A), inv(A) * A, A * inv(A));

x = F7(3)
y = F7(5)
zero(x) = F7(0)
one(x) = F7(1)
+x = F7(3)
-x = F7(4)
x + y = F7(1)
x - y = F7(5)
x * y = F7(1)
x / y = F7(2)
x \ y = F7(4)
x ^ 3 = F7(6)
x ^ -5 = F7(3)
x == F7(3) = true
x == 3 = true

A = F{7, BigInt}[F7(1) F7(2); F7(3) F7(4)]
4A = F{7, BigInt}[F7(4) F7(1); F7(5) F7(2)]
A / 4 = F{7, BigInt}[F7(2) F7(4); F7(6) F7(1)]

L = F{7, BigInt}[F7(1) F7(0); F7(3) F7(1)]
U = F{7, BigInt}[F7(1) F7(2); F7(0) F7(5)]
L * U = F{7, BigInt}[F7(1) F7(2); F7(3) F7(4)]
det(A) = F7(5)
inv(A) = F{7, BigInt}[F7(5) F7(1); F7(5) F7(3)]
inv(A) * A = F{7, BigInt}[F7(1) F7(0); F7(0) F7(1)]
A * inv(A) = F{7, BigInt}[F7(1) F7(0); F7(0) F7(1)]


In [2]:
"""See https://docs.julialang.org/en/v1/manual/interfaces/"""
Base.iterate(Fp::Type{F{p, T}}) where {p, T<:Integer} = (zero(Fp), zero(T))
function Base.iterate(Fp::Type{F{p, T}}, state) where {p, T<:Integer}
    nextstate = state + 1
    nextstate < p ? (Fp(nextstate), nextstate) : nothing
end
Base.IteratorSize(Fp::Type{F{p, T}}) where {p, T<:Integer} = Base.HasLength()
Base.length(Fp::Type{F{p, T}}) where {p, T<:Integer} = p
Base.eltype(Fp::Type{F{p, T}}) where {p, T<:Integer} = Fp

squares(Fp) = Fp[x^2 for x in Fp]
squareroots(k, Fp) = Fp[x for x in Fp if x^2 == k]
@show(collect(F7), squares(F7), squareroots.(0:6, Ref(F7)));

collect(F7) = F{7, BigInt}[F7(0), F7(1), F7(2), F7(3), F7(4), F7(5), F7(6)]
squares(F7) = F{7, BigInt}[F7(0), F7(1), F7(4), F7(2), F7(2), F7(4), F7(1)]
squareroots.(0:6, Ref(F7)) = Vector{F{7, BigInt}}[[F7(0)], [F7(1), F7(6)], [F7(3), F7(4)], [], [F7(2), F7(5)], [], []]


In [3]:
using GaloisFields, LinearAlgebra{}
GF7 = @GaloisField 7
B = GF7[1 2; 3 4]
det(B)

LoadError: syntax: extra token "{" after end of expression

In [4]:
inv(B)

LoadError: UndefVarError: B not defined

In [5]:
lu(B)

LoadError: UndefVarError: B not defined

In [6]:
using AbstractAlgebra

@show GF7 = GF(7)
@show P, x = PolynomialRing(GF7, "x")
@show C = GF7[1 2; 3 4]
squares(Fp) = [x^2 for x in Fp]
squareroots(k, Fp) = [x for x in Fp if x^2 == k]
println()
@show(det(C), inv(C), lu(C), charpoly(P, C))
println()
@show(collect(GF7), squares(GF7), squareroots.(0:6, Ref(GF7)));

GF7 = GF(7) = Finite field F_7
(P, x) = PolynomialRing(GF7, "x") = (Univariate Polynomial Ring in x over Finite field F_7, x)
C = GF7[1 2; 3 4] = [1 2; 3 4]

det(C) = 5
inv(C) = [5 1; 5 3]
lu(C) = (2, (), [1 0; 3 1], [1 2; 0 5])
charpoly(P, C) = x^2 + 2*x + 5

collect(GF7) = AbstractAlgebra.GFElem{Int64}[0, 1, 2, 3, 4, 5, 6]
squares(GF7) = AbstractAlgebra.GFElem{Int64}[0, 1, 4, 2, 2, 4, 1]
squareroots.(0:6, Ref(GF7)) = Vector{AbstractAlgebra.GFElem{Int64}}[[0], [1, 6], [3, 4], [], [2, 5], [], []]


In [7]:
@which iterate(GF7)

iterate(f::FinField) in AbstractAlgebra at D:\.julia\packages\AbstractAlgebra\hlSYb\src\algorithms\FinField.jl:34

In [8]:
@code_warntype iterate(F7, big(1))

MethodInstance for iterate(::Type{F{7, BigInt}}, ::BigInt)
  from iterate(Fp::Type{F{p, T}}, state) where {p, T<:Integer} in Main at In[2]:3
Static Parameters
  p = 7
  T = BigInt
Arguments
  #self#::Core.Const(iterate)
  Fp::Core.Const(F{7, BigInt})
  state::BigInt
Locals
  nextstate::BigInt
Body::Union{Nothing, Tuple{F{7, BigInt}, BigInt}}
1 ─      (nextstate = state + 1)
│   %2 = (nextstate < $(Expr(:static_parameter, 1)))::Bool
└──      goto #3 if not %2
2 ─ %4 = (Fp)(nextstate)::F{7, BigInt}
│   %5 = Core.tuple(%4, nextstate)::Tuple{F{7, BigInt}, BigInt}
└──      return %5
3 ─      return Main.nothing



In [9]:
[x for x in F7 if x^2 == 4]

2-element Vector{F{7, BigInt}}:
 F7(2)
 F7(5)

In [10]:
[x for x in F7 if x^2 == 5]

Any[]

In [11]:
@which collect(x for x in F7 if x^2 == 5)

collect(itr::Base.Generator) in Base at array.jl:704

In [12]:
Base.@default_eltype(x for x in F7 if x^2 == 5)

Any

In [13]:
Base.@default_eltype(x for x in F7 if x^2 == 5)

Any

In [14]:
(@macroexpand Base.@default_eltype(x for x in F7 if x^2 == 5)) |> Base.remove_linenums!

quote
    if (x for x = F7 if x ^ 2 == 5) isa Base.Generator && ((x for x = F7 if x ^ 2 == 5)).f isa Base.Type
        ((x for x = F7 if x ^ 2 == 5)).f
    else
        (Base.Core).Compiler.return_type(Base._iterator_upper_bound, Base.Tuple{Base.typeof((x for x = F7 if x ^ 2 == 5))})
    end
end

In [15]:
iter = (x for x in F7 if x^2 == 5)

Base.Generator{Base.Iterators.Filter{var"#29#30", DataType}, typeof(identity)}(identity, Base.Iterators.Filter{var"#29#30", DataType}(var"#29#30"(), F{7, BigInt}))

In [16]:
Base._iterator_upper_bound(iter)

LoadError: nothing

In [17]:
Core.Compiler.return_type(Base._iterator_upper_bound, Tuple{typeof(iter)})

Any

In [18]:
@which Base.inferencebarrier(nothing)

inferencebarrier(x) in Base at essentials.jl:756

In [19]:
iter2 = (x for x in F7 if x^2 == 4)

Base.Generator{Base.Iterators.Filter{var"#31#32", DataType}, typeof(identity)}(identity, Base.Iterators.Filter{var"#31#32", DataType}(var"#31#32"(), F{7, BigInt}))

In [20]:
Base._iterator_upper_bound(iter2)

LoadError: TypeError: non-boolean (Nothing) used in boolean context

In [21]:
iter3 = (x for x in GF7 if x^2 == 4)
collect(iter3)

2-element Vector{AbstractAlgebra.GFElem{Int64}}:
 2
 5

In [22]:
iter4 = (x for x in GF7 if x^2 == 5)
collect(iter4)

AbstractAlgebra.GFElem{Int64}[]

In [23]:
Core.Compiler.return_type(Base._iterator_upper_bound, Tuple{typeof(iter4)})

AbstractAlgebra.GFElem{Int64}

In [24]:
Base._iterator_upper_bound(iter4)

LoadError: nothing

In [25]:
@code_warntype Base._iterator_upper_bound(iter4)

MethodInstance for Base._iterator_upper_bound(::Base.Generator{Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}, typeof(identity)})
  from _iterator_upper_bound(itr) in Base at array.jl:661
Arguments
  #self#::Core.Const(Base._iterator_upper_bound)
  itr::Base.Generator{Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}, typeof(identity)}
Locals
  x::Union{Nothing, Tuple{AbstractAlgebra.GFElem{Int64}, AbstractAlgebra.FinFieldIterator{AbstractAlgebra.GFElem{Int64}}}}
  val::AbstractAlgebra.GFElem{Int64}
Body::AbstractAlgebra.GFElem{Int64}
1 ─      (x = Base.iterate(itr))
2 ┄ %2 = (x !== Base.nothing)::Bool
└──      goto #6 if not %2
3 ─      (val = Base.getfield(x::Tuple{AbstractAlgebra.GFElem{Int64}, AbstractAlgebra.FinFieldIterator{AbstractAlgebra.GFElem{Int64}}}, 1))
│   %5 = Base.inferencebarrier(Base.nothing)::Any
└──      goto #5 if not %5
4 ─      return val
5 ─ %8 = Base.getfield(x::Tuple{AbstractAlgebra.GFElem{Int64}, AbstractAlgebra.FinFieldIt

In [26]:
@code_warntype Base._iterator_upper_bound(iter2)

MethodInstance for Base._iterator_upper_bound(::Base.Generator{Base.Iterators.Filter{var"#31#32", DataType}, typeof(identity)})
  from _iterator_upper_bound(itr) in Base at array.jl:661
Arguments
  #self#::Core.Const(Base._iterator_upper_bound)
  itr::Base.Generator{Base.Iterators.Filter{var"#31#32", DataType}, typeof(identity)}
Locals
  x::Union{Nothing, Tuple{Any, Any}}
  val::Any
Body::Any
1 ─      (x = Base.iterate(itr))
2 ┄ %2 = (x !== Base.nothing)::Bool
└──      goto #6 if not %2
3 ─      (val = Base.getfield(x::Tuple{Any, Any}, 1))
│   %5 = Base.inferencebarrier(Base.nothing)::Any
└──      goto #5 if not %5
4 ─      return val
5 ─ %8 = Base.getfield(x::Tuple{Any, Any}, 2)::Any
│        (x = Base.iterate(itr, %8))
└──      goto #2
6 ─      Base.throw(Base.nothing)
└──      Core.Const(:(return %11))



In [27]:
@code_warntype iterate(iter4)

MethodInstance for iterate(::Base.Generator{Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}, typeof(identity)})
  from iterate(g::Base.Generator, s...) in Base at generator.jl:42
Arguments
  #self#::Core.Const(iterate)
  g::Base.Generator{Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}, typeof(identity)}
  s::Tuple{}
Locals
  y::Union{Nothing, Tuple{AbstractAlgebra.GFElem{Int64}, AbstractAlgebra.FinFieldIterator{AbstractAlgebra.GFElem{Int64}}}}
Body::Union{Nothing, Tuple{AbstractAlgebra.GFElem{Int64}, AbstractAlgebra.FinFieldIterator{AbstractAlgebra.GFElem{Int64}}}}
1 ─       nothing
│   %2  = Base.getproperty(g, :iter)::Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}
│   %3  = Core.tuple(%2)::Tuple{Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}}
│         (y = Core._apply_iterate(Base.iterate, Base.iterate, %3, s))
│   %5  = (y === Base.nothing)::Bool
└──       goto #3 if not %5
2 ─       return Base.nothing


In [28]:
@code_warntype iterate(iter)

MethodInstance for iterate(::Base.Generator{Base.Iterators.Filter{var"#29#30", DataType}, typeof(identity)})
  from iterate(g::Base.Generator, s...) in Base at generator.jl:42
Arguments
  #self#::Core.Const(iterate)
  g::Base.Generator{Base.Iterators.Filter{var"#29#30", DataType}, typeof(identity)}
  s::Tuple{}
Locals
  y::Union{Nothing, Tuple{Any, Any}}
Body::Union{Nothing, Tuple{Any, Any}}
1 ─       nothing
│   %2  = Base.getproperty(g, :iter)::Base.Iterators.Filter{var"#29#30", DataType}
│   %3  = Core.tuple(%2)::Tuple{Base.Iterators.Filter{var"#29#30", DataType}}
│         (y = Core._apply_iterate(Base.iterate, Base.iterate, %3, s))
│   %5  = (y === Base.nothing)::Bool
└──       goto #3 if not %5
2 ─       return Base.nothing
3 ─ %8  = y::Tuple{Any, Any}
│   %9  = Core.apply_type(Base.Tuple, Base.Any, Base.Any)::Core.Const(Tuple{Any, Any})
│         (y = Core.typeassert(%8, %9))
│   %11 = Base.getproperty(g, :f)::Core.Const(identity)
│   %12 = Base.getindex(y::Tuple{Any, Any}, 1)::

In [29]:
Core._apply_iterate(Base.iterate, Base.iterate, Core.tuple(iter3.iter), ())

(2, AbstractAlgebra.FinFieldIterator{AbstractAlgebra.GFElem{Int64}}(AbstractAlgebra.GFElem{Int64}[1], [2]))

In [30]:
Core._apply_iterate(Base.iterate, Base.iterate, Core.tuple(iter2.iter), ())

(F7(2), 2)

In [31]:
Core.tuple(iter.iter)

(Base.Iterators.Filter{var"#29#30", DataType}(var"#29#30"(), F{7, BigInt}),)

In [32]:
Core.tuple(iter4.iter)

(Base.Iterators.Filter{var"#35#36", AbstractAlgebra.GFField{Int64}}(var"#35#36"(), Finite field F_7),)